# Demo grid object with functions

In [1]:
import pandas as pd
import pandapower as pp
import os,sys,inspect
import matplotlib
import matplotlib.pyplot as pyplt
import plotly.graph_objs as go
import datetime
from datetime import date, timedelta
import copy
currentdir = os.path.abspath(os.getcwd())
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)
from capacitymap.analysis import analysis_check
from capacitymap import controllers
analysis_check.check_violations.counter=0
from capacitymap.grid import grid



## Create grid object and plot grid at normal operation

In [2]:
#create a grid instance for testing
empt_df = pd.DataFrame(columns=['Start', 'End', 'Object_type','ObjectID', 'Status'])
net = pp.from_json( os.path.join(currentdir,'data\svedala\svedala.json'))
network = grid.Grid('test',net, {}, empt_df)
go.Figure(network.plot_traces())

In [4]:
network.project_df #empty tabel

,Start,End,Object_type,ObjectID,Status


In [5]:
network.config_dict #no configuration changes planned

{}

## Store new project

In [6]:
#new incoming project, here one line will be taken out-of-service for a time period
new_request = {'Start':datetime.datetime(2021,9,1), 
                'End':datetime.datetime(2021,9,3), 
                'Object_type':'line',
                'ObjectID':network.grid.line.name.loc[3], 
                'Status':False}

new_request2 = {'Start':datetime.datetime(2021,9,5), 
                'End':datetime.datetime(2021,9,15), 
                'Object_type':'line',
                'ObjectID':network.grid.line.name.loc[9], 
                'Status':False}

#project is stored
network.store_project_and_update_config(new_request)
network.store_project_and_update_config(new_request2)

In [7]:
network.project_df


,Start,End,Object_type,ObjectID,Status
0,2021-09-01,2021-09-03,line,None,False
1,2021-09-05,2021-09-15,line,None,False


In [8]:
#a record for every timestep, here 1 day is created for the stored project and the configuration change is stored on every timestep
network.config_dict

{datetime.datetime(2021, 9, 15, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 14, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 1, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 6, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 9, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 7, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 2, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 5, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 8, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 12, 0, 0): [{'Ob

## Get grid model for specific date

In [9]:
date = datetime.datetime(2021,9,1)
network.active_config = network.config_dict[date]
network.config()
go.Figure(network.plot_traces())

In [10]:
network.restore() #restore grid mdel back to normal grid

## Remove planned project

In [11]:
project_to_drop = new_request

In [12]:
network.remove_project_and_update_config(project_to_drop)

In [13]:
network.project_df

,Start,End,Object_type,ObjectID,Status
1,2021-09-05,2021-09-15,line,None,False


In [14]:
network.config_dict

{datetime.datetime(2021, 9, 15, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 14, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 6, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 9, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 7, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 5, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 8, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 12, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 10, 0, 0): [{'Object_type': 'line',
   'ObjectID': None,
   'Status': False}],
 datetime.datetime(2021, 9, 11, 0, 0): [{'

## Test new request before saving

In [15]:
new_proj = {'Start':datetime.datetime(2021,9,1), 
                'End':datetime.datetime(2021,9,3), 
                'Object_type':'line',
                'ObjectID':network.grid.line.name.loc[4], 
                'Status':False}
#Generate dict of configuration related to new project
conf1 = grid.generate_config_from_project(new_proj)
conf1

{datetime.datetime(2021, 9, 1, 0, 0): {'Object_type': 'line',
  'ObjectID': None,
  'Status': False},
 datetime.datetime(2021, 9, 2, 0, 0): {'Object_type': 'line',
  'ObjectID': None,
  'Status': False},
 datetime.datetime(2021, 9, 3, 0, 0): {'Object_type': 'line',
  'ObjectID': None,
  'Status': False}}

In [16]:
#combine new project dict with already saved projects from config_dict
config_to_test = grid.combine_dict(conf1,network.config_dict)

# select active config based on time and config_dict with approved plans and new plan
test_date = datetime.datetime(2021,9,1)
network.active_config = config_to_test[test_date]
network.config()

# Analyse active config
analysis_res, exp = analysis_check.check_violations(network.grid)
print(analysis_res)

# Restore to normal before proceeding
network.restore()

(True, True, True, True, False, False, False)
